In [10]:
import pickle
import pandas as ps
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer

import sys
sys.path.append("..")

In [11]:
from src.datasets import QuestionAnswerDataset
from src.models import PTM

In [6]:
with open("../data/folds/tgkf_valid_00.pkl", "rb") as f:
    df = pickle.load(f)
    
print(df.shape)
df.head(2)

(1216, 41)


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.0,1.0,0.800000,1.0,0.000000,0.0,1.000000
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.0,1.0,0.666667,0.0,0.333333,1.0,0.888889


In [7]:
QUESTION_TARGETS = [
    "question_asker_intent_understanding", 
    "question_body_critical",
    "question_conversational",
    "question_expect_short_answer",
    "question_fact_seeking",
    "question_has_commonly_accepted_answer",
    "question_interestingness_others",
    "question_interestingness_self",
    "question_multi_intent",
    "question_not_really_a_question",
    "question_opinion_seeking",
    "question_type_choice",
    "question_type_compare",
    "question_type_consequence",
    "question_type_definition",
    "question_type_entity",
    "question_type_instructions",
    "question_type_procedure",
    "question_type_reason_explanation",
    "question_type_spelling",
    "question_well_written",
]  # 21 variables
ANSWER_TARGETS = [
    "answer_helpful",
    "answer_level_of_information",
    "answer_plausible",
    "answer_relevance",
    "answer_satisfaction",
    "answer_type_instructions",
    "answer_type_procedure",
    "answer_type_reason_explanation",
    "answer_well_written",
]  # 9 variables
TARGETS = QUESTION_TARGETS + ANSWER_TARGETS  # 30 variables

In [27]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
question_dataset = QuestionAnswerDataset(
    df=df,
    target=QUESTION_TARGETS,
    mode="question",
    train_mode=False,
    tokenizer=bert_tokenizer,
)
answer_dataset = QuestionAnswerDataset(
    df=df,
    target=ANSWER_TARGETS,
    mode="question",
    train_mode=False,
    tokenizer=bert_tokenizer,
)

In [38]:
BATCH_SIZE = 32
DEVICE = torch.device("cuda:0")

question_model = PTM(pretrain_dir="bert-base-uncased", num_classes=21, pad_token=0).to(DEVICE)
question_model.load_state_dict(torch.load("../logs/folds/bert_base_uncased_question_00/checkpoints/best.pth", map_location=DEVICE)["model_state_dict"])
question_model = question_model.eval()

answer_model = PTM(pretrain_dir="bert-base-uncased", num_classes=9, pad_token=0).to(DEVICE)
answer_model.load_state_dict(torch.load("../logs/folds/bert_base_uncased_answer_00/checkpoints/stage1.6.pth", map_location=DEVICE)["model_state_dict"])
answer_model = answer_model.eval()

In [39]:
def to_device(tensors, device):
    return {k: t.to(device) for k, t in tensors.items()}


def make_preds(model, dataset, batch_size):
    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False
    )
    preds = []
    with torch.no_grad():
        for inputs in loader:
            out = model(inputs["sequences"].to(DEVICE))
            out = torch.sigmoid(out)
            preds.append(out.detach().cpu().numpy())
    preds = np.vstack(preds)
    return preds

In [40]:
question_preds = make_preds(question_model, question_dataset, BATCH_SIZE)
print(question_preds.shape)

(1216, 21)


In [41]:
answer_preds = make_preds(answer_model, answer_dataset, BATCH_SIZE)
print(answer_preds.shape)

(1216, 9)


In [47]:
preds = ps.DataFrame(np.zeros((question_preds.shape[0], len(TARGETS))), columns=TARGETS)
preds["qa_id"] = df["qa_id"].values
preds[QUESTION_TARGETS] = question_preds
preds[ANSWER_TARGETS] = answer_preds
preds.head()

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,qa_id
0,0.947325,0.678187,0.166771,0.650530,0.468952,0.319037,0.678908,0.578742,0.490474,0.004765,...,0.876851,0.642697,0.935332,0.927269,0.740443,0.121125,0.054681,0.466612,0.921638,0
1,0.912703,0.413801,0.015182,0.775522,0.743178,0.875024,0.575993,0.475409,0.564825,0.006832,...,0.869604,0.614643,0.919763,0.908876,0.732653,0.317341,0.169198,0.733738,0.896899,2
2,0.924167,0.807801,0.002930,0.768715,0.905622,0.951707,0.566016,0.508973,0.198006,0.001406,...,0.877471,0.620669,0.936784,0.938362,0.756446,0.081539,0.042061,0.899257,0.882983,5
3,0.759598,0.388568,0.009575,0.697359,0.798398,0.947763,0.500638,0.397775,0.772252,0.008074,...,0.941683,0.728121,0.960863,0.971516,0.889617,0.584060,0.238594,0.660766,0.851673,7
4,0.792298,0.466666,0.008255,0.720517,0.868950,0.851353,0.504415,0.397099,0.196197,0.007954,...,0.570117,0.470108,0.672317,0.663161,0.540576,0.314572,0.441770,0.079004,0.740041,22


In [48]:
from scipy.stats import spearmanr


def spearman(input, target, classes) -> float:
    score = 0.0
    for c in classes:
        score += np.nan_to_num(spearmanr(target[:, c], input[:, c]).correlation) / len(classes)
    return score

In [59]:
spearman(
    input=preds[TARGETS].values, 
    target=df[TARGETS].values, 
    classes=list(range(30))
)

0.3343087727083016

In [60]:
bert_tokenizer.tokenize("hey [SPECIAL TOKEN] you")

['hey', '[', 'special', 'token', ']', 'you']

In [62]:
bert_tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}